In [167]:
!ls data

descriptions_test     features_train        sample_submission.csv
descriptions_train    images_test           tags_test
features_test         images_train          tags_train


In [58]:
import numpy as np
import pandas as pd

import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score as cv
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression as Logit

import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import utils.data_parsers as dp

import random

In [2]:
df = dp.load_dataframe()

In [13]:
df['desc_joined'] = df['descriptions'].apply(lambda x: " ".join(x).split(" "))
df['desc_joined'] = df['desc_joined'].apply(lambda x: [word for word in x if word not in df.columns])

In [134]:
model = gensim.models.KeyedVectors.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
def get_word2vec(df, model, desc_column='descriptions'):
    vecs = df[desc_column].apply(
        lambda x: model.get_vector(x)
    )

### Applying naive bayes to BOW representation of description

In [14]:
mlb = MultiLabelBinarizer()
one_hot_encodings = mlb.fit_transform(df['desc_joined'])
one_hot_columns = mlb.classes_

train_one_hot = df.join(
    pd.DataFrame(
        one_hot_encodings,
        columns=one_hot_columns,
        index=df.index
    ))

train_one_hot = train_one_hot

train_one_hot.head()

,image_file,resnet_vector,descriptions,tags,desc_joined,,"""","""1802"".","""3-way""","""30",...,zoo,"zoo,",zoo.,zoo..,zookeeper,zoom,zucchini,"zucchini,",zucchini.,"zucchinis,"
0,images_train/5373.jpg,"[-0.8994496464729309, -0.9304700493812561, -2....","[a red train is docked at the station, Several...","[vehicle:train, person:person, indoor:clock, a...","[a, red, train, is, docked, at, the, station, ...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,images_train/984.jpg,"[-1.3469539880752563, -3.1194605827331543, -0....",[A man with blue jersey holding a baseball bat...,"[person:person, sports:baseball bat]","[A, man, with, blue, jersey, holding, a, baseb...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,images_train/7127.jpg,"[-3.44549822807312, -1.5245732069015503, -1.00...",[A kitchen decorated in red and white with acc...,"[appliance:refrigerator, appliance:oven, appli...","[A, kitchen, decorated, in, red, and, white, w...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,images_train/9609.jpg,"[1.1146496534347534, -2.1671018600463867, 0.09...",[A black and white dog chasing sheep in a fiel...,"[animal:dog, animal:sheep]","[A, black, and, white, dog, chasing, sheep, in...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,images_train/5293.jpg,"[1.6026496887207031, -1.5058174133300781, 3.02...",[Two bears with their mouths open in the water...,[animal:bear],"[Two, bears, with, their, mouths, open, in, th...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Doc2Vec

In [38]:
descriptions_flattened = [doc.split(" ") for i, docs in enumerate(df['descriptions']) for doc in docs]

In [56]:
train_desc, test_desc = train_test_split(descriptions_flattened)

train_docs = [TaggedDocument(doc, [i]) for i, doc in enumerate(train_desc)]
test_docs = [TaggedDocument(doc, [i]) for i, doc in enumerate(test_desc)]

model = Doc2Vec(vector_size=50, min_count=1, epochs=40)

model.build_vocab(train_docs)

# increase epochs
for epoch in range(3):
    model.train(train_docs, epochs=model.epochs, total_examples=model.corpus_count)
    print("Epoch #{} is complete.".format(epoch+1))

Epoch #1 is complete.
Epoch #2 is complete.
Epoch #3 is complete.


In [57]:
test_documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(test_docs)]

In [70]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_docs) - 1)
inferred_vector = model.infer_vector(test_docs[doc_id])
sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_docs[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_docs[sims[index][0]])))

Test Document (3051): «A river with a red brick bridge over it.»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d50,n5,w5,s0.001,t3):

MOST (32146, 0.6554316878318787): «A baby putting it's hand in a blue and white cake.»

MEDIAN (2612, 0.04442016780376434): «Two men who are skateboarding down a street.»

LEAST (30545, -0.4891502261161804): «A cat wearing a tie, laying on a large soft surface.»

